In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Things to show
98.76% repayment rate

- loan amount distribution, w/ mean and median
- lender count distribution
- lender count of loan by gender
- lender count of loan by country
- lender count of loan by tfidf of use
- lender count of loan by posted month
- map: loan country
- map: lender country
- map: lender to loan, the lending flow
- distribution of features in the chosen model
- how many succeeded (fully funded)?
- how many failed ()
- family

In [51]:
# Load necessary data
lenders = pd.read_csv('data/lenders.csv', delimiter=',')
loans = pd.read_csv('data/loans.csv', delimiter=',')

In [52]:
# clean up extra headers in loans and lenders
loans = loans[loans['activity']!='activity']
lenders = lenders[lenders['lender_id'] != 'lender_id']

## Loan amount distribution

In [69]:
lenders['loan_count'] = lenders['loan_count'].astype(int)

In [ ]:
sns.distplot(lenders['loan_count'])

KeyboardInterrupt: 

In [ ]:
sns.kdeplot(lenders['loan_count'], shade=True)

##Success and faillure from loan's eyes
**Success**: fully-funded, including *paid, in_repayment, funded, defaulted*.

**Failure**: not fully-funded, including *expired, inactive_expired*.

In [63]:
success_amount = loans[(loans['status']=='paid') |
                       (loans['status']=='in_repayment') |
                       (loans['status']=='funded') |
                       (loans['status']=='defaulted')].shape[0]

fail_amount = loans[(loans['status']=='expired') |
                    (loans['status']=='inactive_expired')].shape[0]

success_and_fail = success_amount + fail_amount

success_rate = success_amount / float(success_and_fail)
fail_rate = 1 - success_rate

In [67]:
print success_amount
print success_rate
print fail_amount
print fail_rate

793685
0.960405660172
32721
0.0395943398281


In [60]:
loans[loans['status']=='expired'].shape[0]

21436